In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import math
import csv

# -------------------------
# General hyperparameters (initial values)
# -------------------------
batch_size = 64          # Number of sequences per batch
block_size = 128         # Maximum sequence length
max_iters = 3000         # Total number of iterations
eval_interval = 100      # Evaluate loss every this many iterations
learning_rate = 1e-3
eval_iters = 200         # Number of batches for evaluation average loss
n_head = 4               # Number of heads in multi-head attention
dropout = 0              # Dropout probability (set to 0 here)
# n_embd and n_layer will be adjusted in different configurations

# -------------------------
# Device configuration (MPS or CPU)
# -------------------------
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
torch.manual_seed(1337)

# -------------------------
# Data preprocessing
# -------------------------
with open('pokemon.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    dat = train_data if split == 'train' else val_data
    ix = torch.randint(len(dat) - block_size, (batch_size,))
    x = torch.stack([dat[i:i+block_size] for i in ix])
    y = torch.stack([dat[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

@torch.no_grad()
def estimate_loss_and_ppl(model):
    losses = torch.zeros(eval_iters)
    model.eval()
    for k in range(eval_iters):
        X, Y = get_batch('train')
        _, loss = model(X, Y)
        losses[k] = loss.item()
    avg_loss = losses.mean().item()
    model.train()
    return {"train": avg_loss}, {"train": math.exp(avg_loss)}

# -------------------------
# Model definition section
# -------------------------
class Head(nn.Module):
    """ Single self-attention head """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ Multi-head attention module """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ Feed-forward network """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block """
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)        # (B, T, n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T, n_embd)
        x = tok_emb + pos_emb  # (B, T, n_embd)
        x = self.blocks(x)     # (B, T, n_embd)
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (B, T, vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# -------------------------
# Experiment section: Models with different configurations
# -------------------------
# We define several configurations by adjusting n_embd and n_layer.
configs = [
    {"name": "Config1", "n_layer": 4, "n_embd": 144},
    {"name": "Config2", "n_layer": 5, "n_embd": 160},
    {"name": "Config3",  "n_layer": 6, "n_embd": 176},
    {"name": "Config4",  "n_layer": 7, "n_embd": 256},
    {"name": "Config5",  "n_layer": 8, "n_embd": 384},
]

# CSV file to write logs
csv_filename = "experiment_log.csv"
fieldnames = ["config", "iteration", "loss", "perplexity", "param_tokens", "total_training_tokens", "cumulative_time"]

# Open CSV file for writing (will overwrite if exists)
with open(csv_filename, mode="w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # For each configuration:
    for config in configs:
        print(f"\nRunning experiment: {config['name']}")
        # Update global variables used in model construction
        n_layer = config["n_layer"]
        n_embd = config["n_embd"]
    
        # Reconstruct model (model constructor uses current n_embd, n_layer, etc.)
        model = GPTLanguageModel().to(device)
        total_params = sum(p.numel() for p in model.parameters())
        print(f"Total number of parameters: {total_params} ({total_params/1e6:.2f} M)")
    
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        start_time = time.time()
        cumulative_tokens = 0  # total training tokens processed
        
        # Training loop for current configuration
        for iter in range(max_iters):
            xb, yb = get_batch('train')
            cumulative_tokens += xb.numel()  # batch_size * block_size tokens per iteration
            logits, loss = model(xb, yb)
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()
            
            # At evaluation points, write log entry directly to CSV
            if iter % eval_interval == 0 or iter == max_iters - 1:
                losses, perplexities = estimate_loss_and_ppl(model)
                current_time = time.time() - start_time
                log_entry = {
                    "config": config["name"],
                    "iteration": iter,
                    "loss": losses["train"],
                    "perplexity": perplexities["train"],
                    "param_tokens": total_params,  # total number of parameters
                    "total_training_tokens": cumulative_tokens,
                    "cumulative_time": current_time
                }
                writer.writerow(log_entry)
                csvfile.flush()  # Ensure immediate write to disk
                print(f"Iter {iter}: Loss = {losses['train']:.4f}, Perplexity = {perplexities['train']:.4f}, "
                      f"Tokens = {cumulative_tokens}, Time = {current_time:.2f} sec")
                
        # End of current configuration: record final metrics (if not already recorded)
        current_time = time.time() - start_time
        losses, perplexities = estimate_loss_and_ppl(model)
        final_log_entry = {
            "config": config["name"],
            "iteration": max_iters - 1,
            "loss": losses["train"],
            "perplexity": perplexities["train"],
            "param_tokens": total_params,
            "total_training_tokens": cumulative_tokens,
            "cumulative_time": current_time
        }
        writer.writerow(final_log_entry)
        csvfile.flush()
        print(f"{config['name']}: Total time = {current_time:.2f} sec, Final Loss = {losses['train']:.4f}, Perplexity = {perplexities['train']:.4f}")

print(f"\nExperiment log saved to {csv_filename}")

# -------------------------
# Generate sample text using the last trained model
# -------------------------
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # Initial token 0
generated_idx = model.generate(context, max_new_tokens=500)[0].tolist()
print("\nGenerated text:")
print(decode(generated_idx))



Running experiment: Config1
Total number of parameters: 1042061 (1.04 M)
Iter 0: Loss = 3.9410, Perplexity = 51.4724, Tokens = 8192, Time = 13.10 sec
Iter 100: Loss = 2.3898, Perplexity = 10.9108, Tokens = 827392, Time = 45.57 sec
Iter 200: Loss = 2.0521, Perplexity = 7.7840, Tokens = 1646592, Time = 77.24 sec
Iter 300: Loss = 1.6708, Perplexity = 5.3162, Tokens = 2465792, Time = 111.16 sec
Iter 400: Loss = 1.4847, Perplexity = 4.4135, Tokens = 3284992, Time = 143.82 sec
Iter 500: Loss = 1.3743, Perplexity = 3.9523, Tokens = 4104192, Time = 176.01 sec
